In [1]:
import pandas as pd
from datetime import date
import requests

In [2]:
import chtools as ch

#### get_api_key

In [3]:
api_key = ch.get_api_key()
api_key

****API KEY****

#### get_company_data

In [4]:
RegisteredOffice = ch.RegisteredOffice
CompanyData = ch.CompanyData
print(RegisteredOffice)
print(CompanyData)

<class 'chtools.Company_Info.RegisteredOffice'>
<class 'chtools.Company_Info.CompanyData'>


In [5]:
registered_office = [c[0] for c in RegisteredOffice.construct()]
registered_office

['address_line_1',
 'address_line_2',
 'postal_code',
 'locality',
 'region',
 'country']

In [6]:
company_data = [c[0] for c in CompanyData.construct()]
company_data

['company_number',
 'company_name',
 'company_status',
 'date_of_creation',
 'registered_office_address',
 'sic_codes',
 'type']

In [7]:
# Now you can create an empty instance
# Using a loop
company_without_data = [{c[0]:c[1]} for c in CompanyData.construct()]
company_without_data

[{'company_number': None},
 {'company_name': None},
 {'company_status': None},
 {'date_of_creation': None},
 {'registered_office_address': None},
 {'sic_codes': []},
 {'type': None}]

In [8]:
# Or create with some data
company_with_data = CompanyData(
    company_number="12345678",
    company_name="Test Company",
    company_status="active",
    date_of_creation=date(2020, 1, 1),
    registered_office_address=RegisteredOffice(
        address_line_1="123 Test St",
        locality="London",
        postal_code="SW1A 1AA",
        country="United Kingdom"
    )
)
for field, value in company_with_data:
    print(f"{field:<25} {value}")

company_number            12345678
company_name              Test Company
company_status            active
date_of_creation          2020-01-01
registered_office_address address_line_1='123 Test St' address_line_2=None postal_code='SW1A 1AA' locality='London' region=None country='United Kingdom'
sic_codes                 []
type                      None


#### get_constants_from_Config_class

In [18]:
Config = ch.Config(api_key)

# Get all class attributes including constants
class_values = {attr: getattr(Config, attr) for attr in dir(Config) 
                if not attr.startswith('__') and not callable(getattr(Config, attr))}
# Get instance attributes
instance_values = Config.__dict__  # For your specific instance

# Combine both
all_values = {**class_values, **instance_values}
print(all_values)

{'BASE_URL': 'https://api.companieshouse.gov.uk', 'INSURANCE_SIC_CODES': ['651', '652'], 'ITEMS_PER_PAGE': 100, 'MAX_REQUESTS': 590, 'MAX_RESULTS': 1000, 'OUTPUT_DIR': PosixPath('insurance_companies_data'), 'SEARCH_TERMS': ['insurance company', 'insurance limited', 'insurance ltd', 'insurance plc', 'assurance company', 'assurance limited', 'assurance ltd', 'assurance plc'], 'TIME_WINDOW': 300, 'api_key': ****API KEY****, 'arbitrary_types_allowed': True}


In [19]:
class_values


{'BASE_URL': 'https://api.companieshouse.gov.uk',
 'INSURANCE_SIC_CODES': ['651', '652'],
 'ITEMS_PER_PAGE': 100,
 'MAX_REQUESTS': 590,
 'MAX_RESULTS': 1000,
 'OUTPUT_DIR': PosixPath('insurance_companies_data'),
 'SEARCH_TERMS': ['insurance company',
  'insurance limited',
  'insurance ltd',
  'insurance plc',
  'assurance company',
  'assurance limited',
  'assurance ltd',
  'assurance plc'],
 'TIME_WINDOW': 300,
 'api_key': ****API KEY****,
 'arbitrary_types_allowed': True}

#### setup_logger

In [25]:
logger = ch.setup_logging()
logger

<RootLogger root (INFO)>

#### APIResponse

In [10]:
ch.APIResponse()

APIResponse(items=None, total_results=None, page_number=None, kind='None')

#### CompaniesHouseAPI

In [13]:
# %load CompaniesHouseAPI.py
class CompaniesHouseAPI:

    import logging
    import base64
    import requests
    from typing import Optional, Dict, Any
    from requests.adapters import HTTPAdapter
    from requests.packages.urllib3.util.retry import Retry
    # from .Config import Config
    # from .RateLimiter import RateLimiter
    # from .APIResponse import APIResponse
    # from COMPANIES-HOUSE-PROJECT.python_examples.Fix_Pydantic_nested_models \
    #     import Config, APIResponse, RegisteredOffice, CompanyData
    # import Fix_Pydantic_nested_models
    from chtools.Company_Info import Config, APIResponse, RegisteredOffice, CompanyData
    from chtools.RateLimiter import RateLimiter
    from chtools.get_api_key import get_api_key
    from chtools.setup_logging import setup_logging
    logger = setup_logging()
    api_key = get_api_key()

    """Handler for Companies House API interactions."""

    def __init__(self, api_key: str, logger: logging.Logger, rate_limiter: Optional[RateLimiter] = None, config: Optional[Config] = None):
        self.api_key = api_key
        self.logger = logger
        # Pass logger to RateLimiter
        self.rate_limiter = rate_limiter or RateLimiter(config.MAX_REQUESTS, config.TIME_WINDOW, self.logger)  # If rate limiter not passed in use a default one
        self.session = self._create_session()
        self.config = config if config else Config(api_key=api_key)

        # Create base64 encoded authentication
        auth = base64.b64encode(f"{api_key}:".encode('ascii')).decode('ascii')
        self.headers = {
            'Authorization': f'Basic {auth}',
            'Accept': 'application/json'
        }
        self.logger.debug("CompaniesHouseAPI Initialized")

    @staticmethod
    def _create_session() -> requests.Session:
        """Create a session with retry strategy."""
        session = requests.Session()
        retry_strategy = Retry(
            total=3,
            backoff_factor=1,
            status_forcelist=[429, 500, 502, 503, 504]
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://", adapter)
        return session

    def search_companies(self, search_term: str, start_index: int = 0) -> Optional[APIResponse]:
        """Search for companies using the Companies House API."""
        params = {
            'q': search_term,
            'items_per_page': self.config.ITEMS_PER_PAGE,
            'start_index': start_index,
            'company_status': 'active'
        }

        try:
            self.rate_limiter.wait_if_needed()

            if start_index >= self.config.MAX_RESULTS:
                self.logger.info(f"Reached maximum results limit ({self.config.MAX_RESULTS}) for search term: {search_term}")
                return APIResponse(items=[])

            response = self.session.get(
                f"{self.config.BASE_URL}/search/companies",
                headers=self.headers,
                params=params
            )
            response.raise_for_status()
            
            return APIResponse(**response.json())

        except requests.exceptions.RequestException as e:
            self.logger.error(f"Error making request: {e}")
            return None

    def get_company_details(self, company_number: str) -> Optional[Dict[str, Any]]:
        """Get detailed information for a specific company."""
        try:
            self.rate_limiter.wait_if_needed()

            response = self.session.get(
                f"{self.config.BASE_URL}/company/{company_number}",
                headers=self.headers
            )
            response.raise_for_status()
            return response.json()

        except requests.exceptions.RequestException as e:
            self.logger.error(f"Error fetching details for company {company_number}: {e}")
            return None

Invalid api_key: ch.APIResponse()


In [11]:
logger = ch.setup_logging()
rate_limiter = ch.RateLimiter
ch_api = ch.CompaniesHouseAPI(api_key, logger, rate_limiter)
ch_api

NameError: name 'requests' is not defined